In [ ]:
import numpy as np
import pandas as pd
import sklearn
import random
import matplotlib.pyplot as plt
import sys
%matplotlib inline
sys.setrecursionlimit(10000)

In [ ]:
class Lattice:
    def __init__(self,L,T,start='cold',J=1):
        self.J = J
        self.L = L
        self.N = L*L
        self.T = T
        beta = 1/T
        self.metropolisProb = [np.exp(-2*beta*x) for x in range(2,5,2)]
        self.wolffProb = 1 - np.exp(-2*beta*J)
        if start == 'hot':
            self.sites = np.random.choice((-1,1),(L,L))
        elif start == 'cold':
            self.sites = np.ones((L,L),dtype='int8')
        else:
            print("'{}' is an incorrect start configuration.  Valid choices are 'hot' or 'cold'.".format(start))
            print("Default to 'cold' start")
            self.sites = np.ones((L,L),dtype='int8')
            
    def update_metropolis(self):
        for i in range(self.N):
            x = random.randrange(0,self.L)
            y = random.randrange(0,self.L)
            sumNeighbor  = self.sites[(y+1)%self.L][x]
            sumNeighbor += self.sites[(y-1)%self.L][x]
            sumNeighbor += self.sites[y][(x+1)%self.L]
            sumNeighbor += self.sites[y][(x-1)%self.L]
                
            delta = self.sites[y][x]*sumNeighbor
                                
            if delta <= 0:
                self.sites[y][x] *= -1
            elif random.random()<self.metropolisProb[int(delta/2-1)]:
                self.sites[y][x] *= -1
                            
    def update_wolff(self):
        cluster = np.full((self.L,self.L),False,dtype='bool')
        
        X = random.randrange(0,self.L)
        Y = random.randrange(0,self.L)
        
        def tryAdd(x,y,clusterSpin,cluster):
            if self.sites[y][x] == clusterSpin:
                if random.random() < self.wolffProb:
                    growCluster(x,y,clusterSpin,cluster)
        
        def growCluster(x,y,clusterSpin,cluster):
            self.sites[y][x] *= -1
            cluster[y][x] = True
            
            if cluster[(y+1)%self.L][x] == False:
                tryAdd(x,(y+1)%self.L,clusterSpin,cluster)
            if cluster[(y-1)%self.L][x] == False:
                tryAdd(x,(y-1)%self.L,clusterSpin,cluster)
            if cluster[y][(x+1)%self.L] == False:
                tryAdd((x+1)%self.L,y,clusterSpin,cluster)
            if cluster[y][(x-1%self.L)] == False:
                tryAdd((x-1)%self.L,y,clusterSpin,cluster)
                
        growCluster(X,Y,self.sites[Y][X],cluster)

    def m(self):
        return self.sites.sum()/self.N
    
    def e(self):
        e = 0
        for x in range(self.L):
            for y in range(self.L):
                sumNeighbor  = self.sites[(y+1)%self.L][x]
                sumNeighbor += self.sites[(y-1)%self.L][x]
                sumNeighbor += self.sites[y][(x+1)%self.L]
                sumNeighbor += self.sites[y][(x-1)%self.L]
                
                e += -self.sites[y][x]*sumNeighbor
        return e/(2*self.N)
    
    def __str__(self):
        string = "Mean Magnetization Per Site:\t{}\n".format(self.m())
        string += "Mean Energy Per Site:\t\t{}\n".format(self.e())
        string += str(self.sites)
        return string

In [ ]:
def autocorr(x):
    x = x - x.mean()
    result = np.correlate(x, x, mode='full')
    return result[result.size/2:]
def c(x,T,L):
    return 1/(L**2*T**2)*((x*x).mean()-x.mean()**2)
def bootstrap_c(x,T,L,n=100):
    l = len(x)
    c_rs = []
    for i in range(n):
        x_rs = sklearn.utils.resample(x,n_samples=l)
        c_rs.append(c(x_rs,T,L))
    c_np = np.array(c_rs)
    return np.sqrt((c_np*c_np).mean()-c_np.mean()**2)

In [ ]:
hot  = Lattice(32,2,'hot')
cold = Lattice(32,2,'cold')
d    = []
for i in range(1000):
    hot.update_wolff()
    cold.update_wolff()
    
    d.append((cold.m(),hot.m(),cold.e(),hot.e()))

In [ ]:
df = pd.DataFrame(d,columns=['m_cold','m_hot','e_cold','e_hot'])

In [ ]:
ax = df.plot(y=['m_cold','m_hot'],ylim=(0,1.2),figsize=(10,6),color=('b','r'),title="Thermalization of <m>")
ax.set_xlabel("Simulation Time")
ax.set_ylabel("<m>")
ax.patch.set_facecolor('None')

In [ ]:
ax = df.plot(y=['e_cold','e_hot'],ylim=(-4.2,0),figsize=(10,6),color=('b','r'),title="Wolff Thermalization of <e>")
ax.set_xlabel("Simulation Time")
ax.set_ylabel("<e>")
ax.patch.set_facecolor('None')

In [ ]:
plt.plot(autocorr(df.e_hot[100:])[:10],label="e_hot")
plt.plot(autocorr(df.e_cold[100:])[:10],label="e_cold")
plt.legend()
plt.title("Autocorrelation for Wolff")
plt.ylabel("Integrated ACF")
plt.xlabel("Time Steps")

In [ ]:
d_16    = []
L = 16
t_sim = [1,1.25,1.5,1.75,2,2.1,2.2,2.3,2.4,2.5,2.6,3,3.5]
for t in t_sim:
    print("Running simulation for temperature:\t{}".format(t))
    hot  = Lattice(L,t,'hot')
    cold = Lattice(L,t,'cold')
    for i in range(1000):
        if t < 2.9:
            hot.update_wolff()
            cold.update_wolff()
        else:
            hot.update_metropolis()
            cold.update_metropolis()
        d_16.append((t,cold.m(),i,hot.m(),cold.e(),hot.e()))

In [ ]:
df_16 = pd.DataFrame(d_16,columns=['T','step','m_cold','m_hot','e_cold','e_hot'])

In [ ]:
calculation = []
for i in df_16.groupby('T'):
    T = i[0]
    tdf = i[1]
    calculation.append((T,c(tdf.e_cold.values[100:],T,L),bootstrap_c(tdf.m_cold.values[100:],T,L)))

In [ ]:
c_df = pd.DataFrame(calculation,columns=['T','c','c_err'])

In [ ]:
ax = c_df.plot(x='T',y='c',yerr='c_err',kind='scatter',title='Specific Heat',figsize=(10,6))
ax.patch.set_facecolor('None')

In [ ]:
df_16[df_16.T==2].plot(x='step',y=['e_hot','e_cold'])

In [ ]:
df_16.head()

In [ ]:
d_32  = []
L     = 32
t_sim = [1,1.25,1.5,1.75,2,2.1,2.2,2.3,2.4,2.5,2.6,3,3.5]
for t in t_sim:
    print("Running simulation for temperature:\t{}".format(t))
    hot  = Lattice(L,t,'hot')
    cold = Lattice(L,t,'cold')
    for i in range(1000):
        hot.update_wolff()
        cold.update_wolff()
        d_32.append((t,cold.m(),i,hot.m(),cold.e(),hot.e()))

In [ ]:
df_32 = pd.DataFrame(d_32,columns=['T','step','m_cold','m_hot','e_cold','e_hot'])

In [ ]:
calculation = []
for i in df_32.groupby('T'):
    T = i[0]
    tdf = i[1]
    calculation.append((T,c(tdf.e_hot.values[100:],t,L),bootstrap_c(tdf.m_hot.values[100:],t,L)))

In [ ]:
c_df = pd.DataFrame(calculation,columns=['T','c','c_err'])
ax = c_df.plot(x='T',y='c',yerr='c_err',kind='scatter',title='Specific Heat',figsize=(10,6))
ax.patch.set_facecolor('None')

In [ ]:
df_32[df_32.T==3].plot(y=['e_cold','e_hot'],ylim=(-4.2,0),figsize=(10,6),color=('b','r'),title="Wolff Thermalization of <e>")

In [ ]:
hot.e()

In [ ]:
import time

In [ ]:
sim = Lattice(10,5,'cold')
print(sim)
t0 = time.time()
for i in range(10):
    sim.update_metropolis()
t1 = time.time() 
print(sim)
print("Elapsed Time:\t{:f}".format((t1-t0)/10))

In [ ]:
sim = Lattice(10,5,'cold')
print(sim)
t0 = time.time()
for i in range(10):
    sim.update_wolff()
t1 = time.time()
print(sim)
print("Elapsed Time:\t{:f}".format((t1-t0)/10))